<a href="https://colab.research.google.com/github/sungshin-ai/ai_teamplay/blob/main/Inception_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 모듈

In [3]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input #InceptionV3, Xception, InceptionResNetV2
#from keras.applications.imagenet_utils import preprocess_input #others
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization

from tensorflow import keras
from PIL import Image
import tensorflow as tf

from keras.callbacks import ModelCheckpoint

## 1. 데이터

In [5]:
select_df=pd.read_csv('/content/drive/MyDrive/DF_select.csv')
select_df

,name,fold,class,num,path,label
0,00_01.0045.jpg,00_01,1,45,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
1,00_01.0046.jpg,00_01,1,46,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
2,00_01.0047.jpg,00_01,1,47,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
3,00_01.0048.jpg,00_01,1,48,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
4,00_01.0049.jpg,00_01,1,49,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
...,...,...,...,...,...,...
3595,19_77.0072.jpg,19_77,77,72,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3596,19_77.0073.jpg,19_77,77,73,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3597,19_77.0074.jpg,19_77,77,74,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3598,19_77.0075.jpg,19_77,77,75,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care


In [6]:
select_df['path']='/content/drive/MyDrive/KSL_rgb/'+select_df['fold']+'/'+select_df['name']

In [7]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
train_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

Found 2700 validated image filenames belonging to 10 classes.
Found 900 validated image filenames belonging to 10 classes.


# InceptionV3

In [8]:
InceptionV3(
    include_top=True, # classification (softmax) 부분을 포함할 것인지 선택하는 것입니다.
    weights="imagenet", # imageNet으로 pre-trained 된 모델을 받을 것인지를 선택하는 것입니다. 만약 pre-trained 된 모델을 받길 원하지 않는다면 None으로 넣으면 됩니다.
    input_tensor=None, 
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

96112376/96112376 [==============================] - 0s 0us/step


## 2. 모델 학습/검증

In [9]:
base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg') #Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5

#base_model.summary()

net = Dense(units=10)(base_model.output)
net = Activation('softmax')(net)
model = Model(inputs=base_model.input, outputs=net)

model.summary()

87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_94 (Conv2D)             (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_94 (BatchN  (None, None, None,   96         ['conv2d_94[0][0]']              
 ormalization)               

In [10]:
checkpoint_filepath = '/content/drive/MyDrive/keras/checkpoint/best_inc.ckpt'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [11]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [12]:
history = model.fit(
          train_generator,
          epochs=10,
          validation_data=valid_generator, 
          callbacks=[model_checkpoint_callback]
)

Epoch 1/10
22/22 [==============================] - ETA: 0s - loss: 1.0233 - accuracy: 0.6467 
Epoch 1: val_accuracy improved from -inf to 0.11111, saving model to /content/drive/MyDrive/keras/checkpoint/best_inc.ckpt
22/22 [==============================] - 950s 42s/step - loss: 1.0233 - accuracy: 0.6467 - val_loss: 3.0669 - val_accuracy: 0.1111
Epoch 2/10
22/22 [==============================] - ETA: 0s - loss: 0.2109 - accuracy: 0.9400
Epoch 2: val_accuracy improved from 0.11111 to 0.11444, saving model to /content/drive/MyDrive/keras/checkpoint/best_inc.ckpt
22/22 [==============================] - 30s 1s/step - loss: 0.2109 - accuracy: 0.9400 - val_loss: 13.4772 - val_accuracy: 0.1144
Epoch 3/10
22/22 [==============================] - ETA: 0s - loss: 0.1444 - accuracy: 0.9585
Epoch 3: val_accuracy did not improve from 0.11444
22/22 [==============================] - 29s 1s/step - loss: 0.1444 - accuracy: 0.9585 - val_loss: 22.7723 - val_accuracy: 0.1111
Epoch 4/10
22/22 [========